In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np

from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [2]:
train = pd.read_csv('train_base.csv', sep=';')
test = pd.read_csv('test_base.csv', sep=';')

In [3]:
bureau = pd.read_csv('bureau.csv')

In [4]:
bureau_balance = pd.read_csv('bureau_balance.csv')

In [5]:
import aggregation as agt

In [6]:
# Аргрегирование:
features_min = ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'AMT_CREDIT_SUM']
features_max = ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'CREDIT_DAY_OVERDUE', 'AMT_CREDIT_SUM', 
                'AMT_CREDIT_SUM_DEBT', 'AMT_ANNUITY']
features_mean = ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_CREDIT_UPDATE', 'CREDIT_DAY_OVERDUE',
                'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_OVERDUE',
                'AMT_CREDIT_SUM_LIMIT', 'AMT_ANNUITY']
features_sum = ['CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_DEBT',
              'AMT_CREDIT_SUM']

In [ ]:
agr_min = agt.aggregate_min_value(bureau, features_min)

In [ ]:
agr_max = agt.aggregate_min_value(bureau, features_max)

In [ ]:
agr_mean = agt.aggregate_min_value(bureau, features_mean)

In [ ]:
agr_sum = agt.aggregate_min_value(bureau, features_sum)

In [34]:
agr_min.to_csv('agr_min', sep=';')
agr_max.to_csv('agr_max', sep=';')
agr_mean.to_csv('agr_mean', sep=';')
agr_sum.to_csv('agr_sum', sep=';')

In [7]:
agr_min = pd.read_csv('agr_min', sep=';')
agr_max = pd.read_csv('agr_max', sep=';')
agr_mean = pd.read_csv('agr_mean', sep=';')
agr_sum = pd.read_csv('agr_sum', sep=';')

In [ ]:
#'MONTHS_BALANCE_MIN': ['min'],
#'MONTHS_BALANCE_MAX': ['max'],
#'MONTHS_BALANCE_SIZE': ['mean', 'sum']

## Категориальные признаки:
### Агрегирование:
#### Для агрегирования категориальных признаков можно попробовать создать признаки для каждого значения категориальных фичей (признак_категория_i) и присвоить ему количество данных категорий для каждого клиента. Также можно упорядочить статус кредита и подсчитать среднее значение (не стоит считать сумму, так как у всех клиентов разное количество строк и результаты будут искажаться)

#### CREDIT_ACTIVE - статус кредита

#### CREDIT_CURRENCY - валюта кредита

#### CREDIT_TYPE - тип кредита 


## Числовые признаки: 
### Агрегирование:
#### Для агрегирования числовых признаков можно создавать признаки с минимальным, средним и максимальным значение для каждого клиента. 

#### SK_ID_CURR - номер клиента

#### SK_ID_BUREAU - номер заявки

#### DAYS_CREDIT - количество прошедших дней с подачи заявки в отношении текущей заявки

#### CREDIT_DAY_OVERDUE - количество просроченных дней

#### DAYS_CREDIT_ENDDATE - количество дней, в течении которых кредит действителен 

#### DAYS_ENDDATE_FACT - количество дней с момента окончания срока кредита 

#### AMT_CREDIT_MAX_OVERDUE - максимальная задолженность по кредиту

#### CNT_CREDIT_PROLONG - сколько раз была продлена дата до погашения

#### AMT_CREDIT_SUM - сумма кредитов

#### AMT_CREDIT_SUM_DEBT - сумма задолженностей

#### AMT_CREDIT_SUM_LIMIT - текущий кредит

#### AMT_CREDIT_SUM_OVERDUE - текущий просроченный платеж по кредиту

#### DAYS_CREDIT_UPDATE - количество дней, когда было получено последнее обновление по кредиту

#### AMT_ANNUITY - аннуитет данных кредитных бюро. 

In [8]:
def create_data_aggregation(data, data_agr, feature):
    without_credit = list(set(data[feature]) - set(data_agr[feature]))
    empty_df = pd.DataFrame(np.nan, index=without_credit, columns=list(data_agr))
    empty_df.fillna(9999, inplace=True)
    empty_df['dummy_credit_history'] = 0
    data_agr['dummy_credit_history'] = 1
    features = pd.concat([empty_df, data_agr])
    features[feature] = features.index
    
    return features

In [9]:
with_credit = bureau.loc[bureau.SK_ID_CURR.isin(list(train.SK_ID_CURR.unique()))]

In [10]:
min_df = create_data_aggregation(with_credit, agr_min, 'SK_ID_CURR')

In [11]:
sum_df = create_data_aggregation(with_credit, agr_sum, 'SK_ID_CURR')

In [12]:
max_df = create_data_aggregation(with_credit, agr_max, 'SK_ID_CURR')

In [13]:
mean_df = create_data_aggregation(with_credit, agr_mean, 'SK_ID_CURR')

In [14]:
df1 = train.merge(min_df, on='SK_ID_CURR', how='left')

In [15]:
df2 = df1.merge(sum_df, on='SK_ID_CURR', how='left')

In [16]:
df3 = df2.merge(max_df, on='SK_ID_CURR', how='left')

In [17]:
df4 = df3.merge(mean_df, on='SK_ID_CURR', how='left')

In [ ]:
# С добавлением нижних фичей ничего не поменялось. Пробовала с ними и без них 

In [18]:
df4['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [19]:
df4['DAYS_EMPLOYED_PERC'] = df4['DAYS_EMPLOYED'] / df4['DAYS_BIRTH']
df4['INCOME_CREDIT_PERC'] = df4['AMT_INCOME_TOTAL'] / df4['AMT_CREDIT']
df4['INCOME_PER_PERSON'] = df4['AMT_INCOME_TOTAL'] / df4['CNT_FAM_MEMBERS']
df4['ANNUITY_INCOME_PERC'] = df4['AMT_ANNUITY'] / df4['AMT_INCOME_TOTAL']
df4['PAYMENT_RATE'] = df4['AMT_ANNUITY'] / df4['AMT_CREDIT']

In [44]:
X = df4.drop(columns=['SK_ID_CURR', 'Target'])
y = df4['Target']

In [35]:
df4.shape

(146794, 134)

In [36]:
kf = StratifiedKFold(n_splits=5,  shuffle=True)

In [47]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(mobjective='binary',
                               max_depth=47, learning_rate=0.05,
                               n_estimators=300, class_weight='balanced',
                               subsample=0.8,
                               colsample_bytree=0.8,
                               random_state=42,
                               importance_type='gain',
                               reg_alpha=70,
                               reg_lambda=50)
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

train_score 0.7506274339738196
test_score 0.6855084602074843
train_score 0.7475203116115764
test_score 0.6975765869796837
train_score 0.7470173275080837
test_score 0.6927293501516886
train_score 0.7499252657385642
test_score 0.6881783493499153
train_score 0.7474407770345507
test_score 0.705194495477671


In [ ]:
### Модель либо очень сильно переобучается, либо качество никак не меняется. 

In [ ]:
### Остальной блок не получилось присоединить, так как памяти не хватает. Ошибка memoryerror

In [21]:
with_credit = bureau_balance.loc[bureau_balance.SK_ID_BUREAU.isin(list(with_credit.SK_ID_BUREAU.unique()))]

MemoryError: 

In [11]:
agr_max = agt.aggregate_min_value(bureau_balance, ['MONTHS_BALANCE'], 'SK_ID_BUREAU')

In [12]:
agr_min = agt.aggregate_min_value(bureau_balance, ['MONTHS_BALANCE'], 'SK_ID_BUREAU')

In [13]:
agr_mean = agt.aggregate_min_value(bureau_balance, ['MONTHS_BALANCE'], 'SK_ID_BUREAU')

In [18]:
agr_max['SK_ID_BUREAU'] = agr_max.index
agr_min['SK_ID_BUREAU'] = agr_min.index
agr_mean['SK_ID_BUREAU'] = agr_mean.index

In [21]:
min_df = create_data_aggregation(with_credit, agr_min, 'SK_ID_BUREAU')
max_df = create_data_aggregation(with_credit, agr_max, 'SK_ID_BUREAU')
mean_df = create_data_aggregation(with_credit, agr_mean, 'SK_ID_BUREAU')
df1 = train.merge(min_df, on='SK_ID_BUREAU', how='left')
df3 = df1.merge(max_df, on='SK_ID_BUREAU', how='left')
df4 = df3.merge(mean_df, on='SK_ID_BUREAU', how='left')

In [23]:
X = df4.drop(columns=['SK_ID_CURR', 'Target', 'SK_ID_BUREAU'])
y = df4['Target']

In [ ]:
for train_indices, test_indices in kf.split(X, y):
    X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]
    lgbmc = lgb.LGBMClassifier(mobjective='binary',
                               max_depth=47, learning_rate=0.02,
                               n_estimators=500, class_weight='balanced',
                               subsample=0.8,
                               colsample_bytree=0.8,
                               random_state=42,
                               importance_type='gain',
                               reg_alpha=70,
                               reg_lambda=50)
    model = lgbmc.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    score_train = roc_auc_score(y_train, y_pred_train)
    print(f'train_score {score_train}')   
    y_pred_val = model.predict(X_val)
    score_val = roc_auc_score(y_val, y_pred_val)
    print(f'test_score {score_val}')

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
